In [67]:
import requests
from bs4 import BeautifulSoup
from bs4.element import Tag
from lxml import html
import json
import pandas as pd
import re
from src.movie_class import *

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# class Movie:
#     '''class that stores data for a movie'''
#     def __init__(
#         self,
#         title: str = '',
#         poster_path: str = '',
#         movielensId: str = '',
#         movielens_mean_rating: float = None,
#         movielens_std_rating: float = None,
#         movielens_pct_5s: float = None,
#         tmdbId: str = '',
#         imdbId: str = '',
#         budget: int = None,
#         revenue: int = None,
#         tmdb_popularity: float = None,
#         tmdb_vote_average: float = None,
#         tmdb_vote_count: int = None,
#         release_date: str = '',
#         tmdb_overview: str = '',
#         tmdb_original_ln: str = '',
#         cast: list = [],
#         tmdb_genre_1: str = '',
#         tmdb_genre_2: str = '',
#         tmdb_genre_3: str = '',
#         tmdb_recommendations: list = []
#     ):
        
#         self.title = title,
#         self.poster_path = poster_path,
#         self.movielensId = movielensId,
#         self.movielens_mean_rating = movielens_mean_rating,
#         self.movielens_std_rating = movielens_std_rating,
#         self.movielens_pct_5s = movielens_pct_5s,
#         self.tmdbId = tmdbId,
#         self.imdbId = imdbId,
#         self.budget = budget,
#         self.revenue = revenue,
#         self.tmdb_popularity = tmdb_popularity,
#         self.tmdb_vote_average = tmdb_vote_average,
#         self.tmdb_vote_count = tmdb_vote_count,
#         self.release_date = release_date,
#         self.tmdb_overview = tmdb_overview,
#         self.tmdb_original_ln = tmdb_original_ln,
#         self.cast = cast,
#         self.tmdb_genre_1 = tmdb_genre_1,
#         self.tmdb_genre_2 = tmdb_genre_2,
#         self.tmdb_genre_3 = tmdb_genre_3,
#         self.tmdb_recommendations = tmdb_recommendations

#     def load_tmdb_movie(self, movielensId, movie_id):
#         mv_url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&language=en-US'
#         r_mv = requests.get(mv_url, allow_redirects=False)
#         try:
#             if r_mv.status_code == 200:
#                 pass
#             else:
#                 print(f"Fail {r_mv.status_code}, {mv_url}")

#             mv_soup = BeautifulSoup(r_mv.content, "html")
#             dict = json.loads(mv_soup.get_text())
#             self.title = dict['title']
#             self.poster_path = dict['poster_path']
#             self.movielensId = movielensId,
# #             self.movielens_mean_rating = movielens_mean_rating,
# #             self.movielens_std_rating = movielens_std_rating,
# #             self.movielens_pct_5s = movielens_pct_5s,
#             self.tmdbId = movie_id,
#             self.imdbId = dict['imdb_id'],
#             self.budget = dict['budget'],
#             self.revenue = dict['revenue'],
#             self.tmdb_popularity = dict['popularity'],
#             self.tmdb_vote_average = dict['vote_average'],
#             self.tmdb_vote_count = dict['vote_count'],
#             self.release_date = dict['release_date'],
#             self.tmdb_overview = dict['overview'],
#             self.tmdb_original_ln = dict['original_language'],
        
#             if len(dict['genres']) > 0:
#                 self.tmdb_genre_1 = dict['genres'][0]['name']
#             if len(dict['genres']) > 1:
#                 self.tmdb_genre_2 = dict['genres'][1]['name']
#             if len(dict['genres']) > 2:
#                 self.tmdb_genre_3 = dict['genres'][2]['name']
        
#             # Load cast info
#             cast_url = f'https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={api_key}&language=en-US'
#             r_cast = requests.get(cast_url, allow_redirects=False)
#             if r_cast.status_code == 200:
#                 pass
#             else:
#                 print(f"Fail {r_cast.status_code}, {cast_url}")

#             cast_soup = BeautifulSoup(r_cast.content, "html")
#             cast_dict = json.loads(cast_soup.get_text())
#             self.cast = cast_dict['cast'][:10]

#             #Load TMDB recommendations
#             recs_url = f'https://api.themoviedb.org/3/movie/{movie_id}/recommendations?api_key={api_key}&language=en-US'
#             r_recs = requests.get(recs_url, allow_redirects=False)
#             if r_recs.status_code == 200:
#                 pass
#             else:
#                 print(f"Fail {r_recs.status_code}, {recs_url}")

#             recs_soup = BeautifulSoup(r_recs.content, "html")
#             recs_dict = json.loads(recs_soup.get_text())
#             self.tmdb_recommendations = recs_dict['results'][:20]
#         except Exception as msg:
#             print(self.tmdbId, 'Exception', msg)

        
#     def __repr__(self):
#         return repr(self.__dict__)

#     def __setitem__(self, key, item):
#         self.__dict__[key] = item

#     def __getitem__(self, key):
#         return self.__dict__[key]
        

In [15]:
recs = 5
movie_list = pd.read_csv('data/filtered_links.csv')
time_test = movie_list[:recs]
time_test.head()

,movieId,imdbId,tmdbId,count,mean,std,stat_score
0,1,114709,862.0,57309,3.893708,0.921552,5.276036
1,2,113497,8844.0,24228,3.251527,0.959851,4.691304
2,3,113228,15602.0,11804,3.142028,1.008443,4.654692
3,4,114885,31357.0,2523,2.853547,1.108531,4.516344
4,5,113041,11862.0,11714,3.058434,0.996611,4.553351


In [16]:
movie_id = 880 #antonia
# movie_id = 114750 # Dear White People
# movie_id = 55721 # bridesmaids
# movie_id = 629 #The Usual Suspects
# movie_id = 496243 #Parasite
# movie_id = 50348 # Lincoln Lawyer
# movie_id = 637 #Life is Beautiful
time_test = movie_list[:5]
time_test.head(1)

,movieId,imdbId,tmdbId,count,mean,std,stat_score
0,1,114709,862.0,57309,3.893708,0.921552,5.276036


In [17]:
movies = []
for i in range(len(time_test)):
    curr_movie = Movie()
    curr_movie.load_tmdb_features(str(time_test.iloc[i][0]), str(time_test.iloc[i][2]))
    curr_movie.load_movielens_features(str(time_test.iloc[i][4]), str(time_test.iloc[i][5]))
    movies.append(curr_movie)


In [18]:
with open('data/mv_pkl0_400.pkl', 'rb') as input:
    d1 = pickle.load(input)

In [19]:
with open('data/mv_pkl_401-1400.pkl', 'rb') as input:
    d2 = pickle.load(input)

In [20]:
with open('data/mv_pkl1401_10200.pkl', 'rb') as input:
    d3 = pickle.load(input)

In [21]:
with open('data/mv_pkl_10200_12800.pkl', 'rb') as input:
    d4 = pickle.load(input)

In [22]:
with open('data/mv_pkl_12800_14600.pkl', 'rb') as input:
    d5 = pickle.load(input)

In [23]:
with open('data/mv_pkl_14600_15000.pkl', 'rb') as input:
    d6 = pickle.load(input)

In [37]:
movie_meta = d1[:401] + d2 + d3 + d4 + d5 + d6[1:]

In [25]:
with open('data/mv_pkl.pkl','wb') as file:
    pickle.dump(movie_meta, file)

In [100]:
# Converting the Movie class object to a DataFrame--created a 'convert_to_dict' method to assist with that.
movie_meta[0].convert_to_dict()

{'title': 'Toy Story',
 'poster_path': '/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg',
 'movielensId': '1.0',
 'movielens_mean_rating': '3.8937077945872383',
 'movielens_std_rating': '0.9215518909655429',
 'tmdbId': '862.0',
 'imdbId': 'tt0114709',
 'budget': 30000000,
 'revenue': 373554033,
 'tmdb_popularity': 47.156,
 'tmdb_vote_average': 7.9,
 'tmdb_vote_count': 12920,
 'release_date': 1995,
 'tmdb_overview': "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",
 'tmdb_original_ln': ('en',),
 'cast': [{'cast_id': 14,
   'character': 'Woody (voice)',
   'credit_id': '52fe4284c3a36847f8024f95',
   'gender': 2,
   'id': 31,
   'name': 'Tom Hanks',
   'order': 0,
   'profile_path': '/xndWFsBlClOJFRdhSt4NBwiPq2o.jpg'},
  {'cast_id': 15,
   'character'

In [102]:
movie_meta_clean = pd.DataFrame.from_records([s.convert_to_dict() for s in movie_meta])

In [104]:
movie_meta_clean.head()

,title,poster_path,movielensId,movielens_mean_rating,movielens_std_rating,tmdbId,imdbId,budget,revenue,tmdb_popularity,tmdb_vote_average,tmdb_vote_count,release_date,tmdb_overview,tmdb_original_ln,cast,tmdb_genre_1,tmdb_genre_2,tmdb_genre_3
0,Toy Story,/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg,1.0,3.8937077945872383,0.9215518909655429,862.0,tt0114709,30000000.0,373554033.0,47.156,7.9,12920.0,1995.0,"Led by Woody, Andy's toys live happily in his ...","(en,)","[{'cast_id': 14, 'character': 'Woody (voice)',...",Animation,Comedy,Family
1,Jumanji,/bdHG5Mo83VPobeZZdlSz0Y7HQHB.jpg,2.0,3.2515271586594023,0.9598514433032206,8844.0,tt0113497,65000000.0,262797249.0,9.027,7.2,7549.0,1995.0,When siblings Judy and Peter discover an encha...,"(en,)","[{'cast_id': 1, 'character': 'Alan Parrish', '...",Adventure,Fantasy,Family
2,Grumpier Old Men,/1FSXpj5e8l4KH6nVFO5SPUeraOt.jpg,3.0,3.142028126058963,1.0084427068162019,15602.0,tt0113228,0.0,0.0,12.893,6.6,209.0,1995.0,A family wedding reignites the ancient feud be...,"(en,)","[{'cast_id': 2, 'character': 'Max Goldman', 'c...",Romance,Comedy,"(,)"
3,Waiting to Exhale,/4wjGMwPsdlvi025ZqR4rXnFDvBz.jpg,4.0,2.85354736424891,1.1085307862214668,31357.0,tt0114885,16000000.0,81452156.0,9.019,6.2,78.0,1995.0,"Cheated on, mistreated and stepped on, the wom...","(en,)","[{'cast_id': 1, 'character': 'Savannah Jackson...",Comedy,Drama,Romance
4,Father of the Bride Part II,/lf9RTErt8BSLQy98aSFblElvsCQ.jpg,5.0,3.058434352057368,0.9966113371988836,11862.0,tt0113041,0.0,76578911.0,12.677,6.2,432.0,1995.0,Just when George Banks has recovered from his ...,"(en,)","[{'cast_id': 1, 'character': 'George Banks', '...",Comedy,"(,)","(,)"


In [50]:
df=pd.DataFrame(columns=cols)
for d in list_of_dicts:
    df.append(d['data'], ignore_index=True)

,0
15197,148430.0
15198,148440.0
15199,148442.0
15200,148448.0
15201,148466.0


In [43]:
movie_meta_df = pd.DataFrame.from_records(movie_meta)

KeyError: 0

In [38]:
movie_meta_df = pd.io.json.json_normalize(movie_meta)

/home/salvir1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


AttributeError: 'Movie' object has no attribute 'values'

In [33]:
movie_meta_df = pd.DataFrame(movie_meta)    

In [34]:
movie_meta_df.head()

,0
0,"{'title': 'Toy Story', 'poster_path': '/uXDfjJ..."
1,"{'title': 'Jumanji', 'poster_path': '/bdHG5Mo8..."
2,"{'title': 'Grumpier Old Men', 'poster_path': '..."
3,"{'title': 'Waiting to Exhale', 'poster_path': ..."
4,"{'title': 'Father of the Bride Part II', 'post..."


In [ ]:
# Movie details from TMDB. Contains oveview, genres, budget, revenue, popularity, vote_average, vote_count, release_date, 

url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&language=en-US'
r = requests.get(url, allow_redirects=False)
if r.status_code == 200:
    print(f"Success {r.status_code}, {url}")

soup = BeautifulSoup(r.content, "html")
soup.prettify()
dict = json.loads(soup.get_text())
dict

In [ ]:
# Recommendations from TMDB---ooh this looks good. Love the recs under Parasite. Want to see many of these.

url = f'https://api.themoviedb.org/3/movie/{movie_id}/recommendations?api_key={api_key}&language=en-US'
r = requests.get(url, allow_redirects=False)
if r.status_code == 200:
    print(f"Success {r.status_code}, {url}")

soup = BeautifulSoup(r.content, "html")
soup.prettify()
dict = json.loads(soup.get_text())
dict['results'][0]
for mv in dict['results']:
    print(mv['popularity'], mv['vote_average'], mv['vote_count']/mv['vote_average'], mv['title'], mv['release_date'], mv['genre_ids'])

In [ ]:
# Similar from TMDB -- not nearly so helpful. Similars for Parasite are movies made in Korea.
# Reviews are also not helpful. They're random and inconsistent.

movie_id = 496243
url = f'https://api.themoviedb.org/3/movie/{movie_id}/similar?api_key={api_key}&language=en-US'
r = requests.get(url, allow_redirects=False)
if r.status_code == 200:
    print(f"Success {r.status_code}, {url}")

soup = BeautifulSoup(r.content, "html")
soup.prettify()
dict = json.loads(soup.get_text())
dict['results'][0]

In [ ]:
# Genre list from TMDB
movie_id = 496243
url = f'https://api.themoviedb.org/3/genre/movie/list?api_key={api_key}&language=en-US'
r = requests.get(url, allow_redirects=False)
if r.status_code == 200:
    print(f"Success {r.status_code}, {url}")

soup = BeautifulSoup(r.content, "html")
jsoup.prettify()
dict = json.loads(soup.get_text())
dict


## NYT reviews. 
- API using the title returns a link to the review page. Not successful with being able to get actual reviews from the web page. 

In [ ]:
# url = f'https://api.nytimes.com/svc/movies/v2/reviews/search.json?query=dear-white-people&api-key={nyt_api_key}'

# header_map = [{"User-Agent": "Mozilla/5.0"}, {"User-Agent": "XY"}, {}]

# r = requests.get(url, allow_redirects=True)
# if r.status_code == 200:
#     print(f"Success {r.status_code}, {url}")
#     soup = BeautifulSoup(r.content, "html")
#     soup.p.string
#     print(soup.prettify())
#     dict = json.loads(soup.p.get_text())
#     url2 = dict['results'][0]['link']['url']
#     for header in header_map:
#         r2 = requests.get(url2, allow_redirects=True)
#         print(r2)
#         if r2.status_code == 200:
#             print(f"Redirect success {r2.status_code}, {url2}")
#             soup2 = BeautifulSoup(r2.content, "html")
#             print(soup2.prettify())
#             json_data = soup2.find_all(type="application/ld+json")
#             for i in range(len(json_data)):
#                 json_soup = json.loads(json_data[i].string)
#                 print(json_soup)